In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

pd.set_option('max_columns', None)

In [59]:
train_df = pd.read_csv('/home/tuts/datasets/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/home/tuts/datasets/house-prices-advanced-regression-techniques/test.csv')

train_df = train_df.drop(labels=['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'Id', 'FireplaceQu'], axis=1)

train_df = train_df.dropna(axis=0)

In [60]:
label_columns = list(train_df.dtypes[train_df.dtypes == object].index)

In [61]:
train_df.isna().sum().nunique()

1

In [44]:
for column in label_columns:
    if train_df[column].nunique() > 6:
        le = LabelEncoder()
        df = pd.DataFrame()
        if train_df[column].dtype == object:
            df = pd.DataFrame(le.fit_transform(train_df[column].astype(str)), columns=[column])
        else:
            df = pd.DataFrame(le.fit_transform(train_df[column], columns=[column]))
        train_df = train_df.drop([column], axis=1)
        train_df = pd.concat([train_df, df], axis=1)
    else:
        oh = pd.get_dummies(train_df[column])
        train_df = train_df.drop([column], axis=1)
        train_df = pd.concat([train_df, oh], axis=1)

train_df.shape

5
MSZoning
object
2
Street
object
4
LotShape
object
4
LandContour
object
1
Utilities
object
5
LotConfig
object
3
LandSlope
object
25
Neighborhood
object
9
Condition1
object
6
Condition2
object
5
BldgType
object
8
HouseStyle
object
5
RoofStyle
object
7
RoofMatl
object
14
Exterior1st
object
16
Exterior2nd
object
4
MasVnrType
object
4
ExterQual
object
4
ExterCond
object
5
Foundation
object
4
BsmtQual
object
4
BsmtCond
object
4
BsmtExposure
object
6
BsmtFinType1
object
6
BsmtFinType2
object
4
Heating
object
5
HeatingQC
object
2
CentralAir
object
5
Electrical
object
4
KitchenQual
object
6
Functional
object
6
GarageType
object
3
GarageFinish
object
5
GarageQual
object
5
GarageCond
object
3
PavedDrive
object
9
SaleType
object
6
SaleCondition
object


(1460, 179)

In [45]:
train_df.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'N', 'P', 'Y', 'SaleType', 'Abnorml', 'AdjLand', 'Alloca', 'Family',
       'Normal', 'Partial'],
      dtype='object', length=179)

In [62]:
train_df.shape

(1094, 75)

In [63]:
l1 = Lasso(normalize=True)
l2 = Ridge(normalize=True, solver='lsqr')
rf = RandomForestRegressor()
ab = AdaBoostRegressor()

In [64]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [65]:
X = train_df.drop(['SalePrice'], axis=1)
y = train_df.SalePrice

print(X.shape)
print(y.shape)

(1094, 74)
(1094,)


In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [67]:
X_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.00000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000
mean,57.882353,69.597386,9991.111111,6.253595,5.596078,1972.130719,1986.237908,107.282353,430.888889,48.356863,603.581699,1082.827451,1155.899346,365.928105,5.035294,1526.862745,0.420915,0.064052,1.581699,0.392157,2.848366,1.032680,6.529412,0.603922,1978.078431,1.860131,497.80915,89.918954,48.413072,22.254902,3.188235,16.071895,2.278431,23.390850,6.360784,2007.792157
std,42.711123,21.786025,9028.005845,1.341447,1.097467,31.424282,20.768378,189.868689,432.573880,164.773665,441.891552,380.978296,362.620782,437.816837,44.110316,513.382641,0.514788,0.250292,0.553598,0.499152,0.781335,0.177913,1.601124,0.621378,26.405846,0.645194,187.16580,121.102208,65.186116,59.531453,29.668255,59.957566,36.486204,168.661811,2.672436,1.344931
min,20.000000,21.000000,1300.000000,2.000000,2.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,105.000000,438.000000,0.000000,0.000000,438.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,1900.000000,1.000000,160.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,60.000000,7500.000000,5.000000,5.000000,1953.000000,1968.000000,0.000000,0.000000,0.000000,256.000000,810.000000,887.000000,0.000000,0.000000,1154.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1959.000000,1.000000,360.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9247.000000,6.000000,5.000000,1976.000000,1996.000000,0.000000,375.000000,0.000000,533.000000,1007.000000,1090.000000,0.000000,0.000000,1470.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1981.000000,2.000000,484.00000,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11184.000000,7.000000,6.000000,2003.000000,2005.000000,169.000000,705.000000,0.000000,847.000000,1314.000000,1382.000000,732.000000,0.000000,1775.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2003.000000,2.000000,595.00000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,182.000000,215245.000000,10.000000,9.000000,2009.000000,2010.000000,1600.000000,2260.000000,1474.000000,2153.000000,3138.000000,3138.000000,1872.000000,572.000000,4676.000000,2.000000,2.000000,3.000000,2.000000,6.000000,2.000000,12.000000,3.000000,2010.000000,4.000000,1390.00000,857.000000,547.000000,330.000000,508.000000,480.000000,648.000000,2500.000000,12.000000,2010.000000


In [70]:
print(X_train.isnull().values.any())
print(y_train.isnull().values.any())

False
False
